Super imposing checkboxes and dotted lines into handwritten dataset which is already cropped

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import os
import random

def add_checkboxes(image, num_checkboxes=1, checkbox_size=50, color=(0, 0, 0), thickness=5):
    """
    Adds checkboxes to the image.
    :param image: Input image
    :param num_checkboxes: Number of checkboxes to add
    :param checkbox_size: Size of the checkbox
    :param color: Color of the checkbox lines
    :param thickness: Thickness of the checkbox lines
    :return: Image with checkboxes
    """
    h, w = image.shape[:2]

    for _ in range(num_checkboxes):
        x = random.randint(0, w - checkbox_size)
        y = random.randint(0, h - checkbox_size)
        top_left = (x, y)
        bottom_right = (x + checkbox_size, y + checkbox_size)
        cv2.rectangle(image, top_left, bottom_right, color, thickness)

    return image

def add_centered_dotted_lines(image, num_lines=2, line_length=200, dot_distance=20, color=(0, 0, 0), thickness=2):
    """
    Adds large dotted lines to the center of the image.
    :param image: Input image
    :param num_lines: Number of dotted lines to add
    :param line_length: Length of each dotted line
    :param dot_distance: Distance between dots
    :param color: Color of the dots
    :param thickness: Thickness of the dots
    :return: Image with dotted lines
    """
    h, w = image.shape[:2]
    center_x, center_y = w // 2, h // 2

    for i in range(num_lines // 2):
        y_offset = i * dot_distance * 2
        for j in range(0, line_length, dot_distance):
            cv2.circle(image, (center_x - line_length // 2 + j, center_y - y_offset), 1, color, thickness)
            cv2.circle(image, (center_x - line_length // 2 + j, center_y + y_offset), 1, color, thickness)

    return image

# Load handwritten images
input_dir = '/content/drive/MyDrive/handwritten_crp_clean'
output_dir = '/content/drive/MyDrive/new_handwritten_super_crp'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)

        # Add checkboxes and centered dotted lines
        image_with_checkboxes = add_checkboxes(image.copy(), num_checkboxes=1, checkbox_size=50, color=(0, 0, 0), thickness=5)
        image_with_dotted_lines = add_centered_dotted_lines(image_with_checkboxes.copy(), num_lines=2, line_length=200, dot_distance=20, color=(0, 0, 0), thickness=2)

        # Save the images
        cv2.imwrite(os.path.join(output_dir, f'modified_{filename}'), image_with_dotted_lines)


pairing the super imposed image with original image

In [ ]:
import os
from PIL import Image

def find_matching_file(filename, search_dir):
    """
    Find a file in the search directory that matches the main part of the filename.
    """
    main_part = filename.replace("modified_", "")
    for search_filename in os.listdir(search_dir):
        if search_filename.endswith(".jpg") or search_filename.endswith(".png"):
            if main_part in search_filename:
                return os.path.join(search_dir, search_filename)
    return None

def load_image_pairs(input_dir, output_dir, save_dir=None):
    input_images = []
    output_images = []

    # Ensure save directory exists if provided
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Iterate through files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            input_path = os.path.join(input_dir, filename)

            # Find the corresponding output file
            output_path = find_matching_file(filename, output_dir)

            # Check if the output file exists before attempting to open it
            if output_path:
                input_image = Image.open(input_path).convert("RGB")
                output_image = Image.open(output_path).convert("RGB")

                input_images.append(input_image)
                output_images.append(output_image)

                # Save the paired image if save_dir is provided
                if save_dir:
                    paired_image = Image.new('RGB', (input_image.width + output_image.width, max(input_image.height, output_image.height)))
                    paired_image.paste(input_image, (0, 0))
                    paired_image.paste(output_image, (input_image.width, 0))

                    save_path = os.path.join(save_dir, f"paired_{os.path.basename(output_path)}")
                    paired_image.save(save_path)
                    print(f"Paired image saved at {save_path}")
            else:
                print(f"Warning: Output file not found for {filename}. Skipping.")

    return input_images, output_images

# Example usage:
input_dir = '/content/drive/MyDrive/handwritten_crp_super'
output_dir = '/content/drive/MyDrive/handwritten_crp'  # Make sure this is the correct directory
save_dir = '/content/drive/MyDrive/train_paired_256_new'

input_images, output_images = load_image_pairs(input_dir, output_dir, save_dir)
